In [14]:
import numpy as np
import pandas as pd
import os
from libs.utils import *
from libs.tf_idf import *

In [6]:
DATASET_PATH = r"E:\Learn Machine Learning\Project\Opinion Classifier\res\dataset\nremove_stopwords.xlsx"

data : pd.DataFrame = read_dataset(DATASET_PATH, "xlsx", header=0)

data.head()

15:21:03 root INFO: Đang đọc dữ liệu từ file E:\Learn Machine Learning\Project\Opinion Classifier\res\dataset\nremove_stopwords.xlsx ...
		at Line 330 [read_dataset() in utils.py, utils]
15:21:04 root INFO: Đọc thành công! 
		at Line 338 [read_dataset() in utils.py, utils]


,repair_words,labels
0,"['tiếng', 'anh', 'thi', 'đầu_vào', 'là', 'khôn...",Neutral
1,"['đã', 'từng', 'là', 'niềm', 'mơ_ước', 'khi', ...",Negative
2,"['học_phí', 'hơi', 'cao']",Negative
3,"['̀', '̀', '̣', '̀', '̀', '̉', '̣', '̂', 'đ', ...",Neutral
4,"['thiết_nghĩ', 'đề', 'thi', 'utc', 'nên', 'đổi...",Positive


In [13]:
labels : list[str] = data["labels"].unique().tolist()
labels.sort(reverse=True)
labels

['Positive', 'Neutral', 'Negative']

In [18]:
import string
import re

string_pattele = string.punctuation + '\n'
string_pattele


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_punctuation(text, except_char: str = None):
    try:
        result_text = "".join(
            char for char in text if char not in string_pattele or (except_char and char == except_char))
        result_text = remove_emoji(result_text)
    except TypeError:
        result_text = None
    return result_text

def filter_none_text_id(documents : pd.Series | list[str]) -> list[int]:
    need_filter : list[str] = [text for text in documents]
    none_text_index : list[int] = []
    for i, text in enumerate(need_filter):
        if (text is None):
            print(str(i) + ": " + str(text))
            none_text_index.append(i)
    
    return none_text_index

In [20]:
data["Tokens"] = data["repair_words"].apply(lambda text: [remove_punctuation(x, '_').split()[0] for x in text.split(',')])

In [29]:
corpus: list[list[str]] = [doc for doc in data["Tokens"]]
labels_corpus: list[str] = data["labels"].to_list()

In [38]:
vocabulary = features_extraction(corpus)
vocabulary = list(vocabulary)
vocabulary.sort()

# 1. Xét n-gram = 1:

Tạo phân phối tần suất:

In [72]:
positive_frequency : dict[str, int] = {}
for token in vocabulary:
    positive_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Positive"].values:
    for token in sen:
        positive_frequency[token] += 1

negative_frequency : dict[str, int] = {}
for token in vocabulary:
    negative_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Negative"].values:
    for token in sen:
        negative_frequency[token] += 1

neutral_frequency : dict[str, int] = {}
for token in vocabulary:
    neutral_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Neutral"].values:
    for token in sen:
        neutral_frequency[token] += 1


In [54]:
positive_frequency

{'0': 2,
 '00': 0,
 '000': 2,
 '000đ': 2,
 '01': 0,
 '02': 5,
 '024': 0,
 '03': 1,
 '0396': 0,
 '04': 1,
 '05': 0,
 '06': 1,
 '068': 1,
 '06đ': 1,
 '07h30': 0,
 '08': 1,
 '0918': 0,
 '0936': 0,
 '0973': 0,
 '0979': 0,
 '0981': 0,
 '0986': 0,
 '0đ': 0,
 '1': 60,
 '10': 19,
 '100': 2,
 '1000': 2,
 '100k': 0,
 '100m': 0,
 '100điểm': 1,
 '101': 0,
 '102a2': 0,
 '1045': 1,
 '10700': 1,
 '108': 1,
 '1096': 0,
 '10d': 1,
 '10k': 1,
 '10ml': 0,
 '10tp': 1,
 '10tr': 1,
 '10đ': 3,
 '11': 3,
 '110': 0,
 '1120': 1,
 '1134': 1,
 '1194': 1,
 '11h': 1,
 '11h30': 1,
 '11r': 1,
 '12': 2,
 '120': 2,
 '1256': 0,
 '127': 0,
 '129m2': 1,
 '12h': 0,
 '12h10': 1,
 '12k': 1,
 '13': 0,
 '139': 1,
 '13m2': 0,
 '13ml': 0,
 '14': 0,
 '144hz': 0,
 '1470': 1,
 '14h30': 0,
 '15': 3,
 '150': 1,
 '15000': 0,
 '15h': 0,
 '16': 1,
 '165hz': 0,
 '16h': 0,
 '17': 2,
 '1700': 1,
 '17h30': 0,
 '17h45': 0,
 '17đ': 0,
 '18': 1,
 '18h': 0,
 '18h30': 0,
 '18h30h': 0,
 '18đ': 0,
 '19': 1,
 '190': 0,
 '1900': 0,
 '1918': 0,
 '192

In [73]:
positive_frequency = dict(sorted(positive_frequency.items(), key=lambda x:x[1], reverse=True))
negative_frequency = dict(sorted(negative_frequency.items(), key=lambda x:x[1], reverse=True))
neutral_frequency = dict(sorted(neutral_frequency.items(), key=lambda x:x[1], reverse=True))
negative_frequency

{'không': 260,
 'trường': 209,
 'có': 204,
 'thì': 174,
 'học': 167,
 'là': 151,
 'mình': 149,
 'mà': 131,
 'đi': 129,
 'xe': 125,
 'được': 107,
 'cũng': 103,
 'lại': 99,
 'vào': 97,
 'còn': 93,
 'em': 92,
 'ra': 89,
 'các': 88,
 'cho': 87,
 'để': 87,
 'bạn': 86,
 'cái': 83,
 'rồi': 83,
 'sinh_viên': 81,
 'của': 80,
 'gì': 80,
 'như': 80,
 'nhưng': 76,
 '1': 68,
 'và': 68,
 'này': 66,
 'phải': 62,
 'làm': 60,
 'năm': 58,
 'người': 55,
 'trong': 52,
 'với': 52,
 'nào': 51,
 'ở': 51,
 'nhiều': 50,
 'mấy': 49,
 'khi': 48,
 'nó': 48,
 'nói': 47,
 'về': 47,
 'đã': 47,
 'cô': 46,
 'đâu': 46,
 'mới': 45,
 'nên': 44,
 'thôi': 42,
 'đến': 42,
 'biết': 41,
 'chưa': 41,
 'nữa': 41,
 'tiền': 41,
 'ạ': 41,
 'chứ': 39,
 'mất': 39,
 'thấy': 39,
 'lên': 38,
 'những': 38,
 't': 38,
 'vẫn': 37,
 'xong': 37,
 'anh': 36,
 'một': 36,
 'rất': 35,
 'thầy': 35,
 'bị': 34,
 'chỉ': 34,
 'luôn': 34,
 'quá': 34,
 'đấy': 34,
 'trước': 33,
 'gửi': 32,
 'lớp': 32,
 'điểm': 30,
 'đó': 30,
 'bảo_vệ': 29,
 'tôi': 29,
 

In [74]:
neutral_frequency

{'không': 440,
 'có': 405,
 'mình': 324,
 'em': 305,
 'là': 304,
 'trường': 277,
 'được': 255,
 'và': 253,
 'ạ': 251,
 'học': 244,
 'bạn': 232,
 'thì': 231,
 'cho': 225,
 'của': 203,
 'các': 188,
 'với': 168,
 'cậu': 161,
 'đã': 132,
 'cũng': 130,
 'mà': 128,
 'đi': 123,
 'lại': 121,
 'để': 119,
 '1': 118,
 'sinh_viên': 115,
 'người': 108,
 'năm': 108,
 'ở': 108,
 'nhưng': 107,
 'hỏi': 104,
 'trong': 102,
 'rồi': 99,
 'đến': 99,
 'còn': 92,
 'này': 91,
 'nào': 90,
 'giao_thông': 89,
 'tôi': 88,
 'vào': 88,
 'về': 88,
 'một': 87,
 'thi': 87,
 'những': 86,
 'phải': 83,
 '2': 82,
 'nhé': 78,
 'gì': 76,
 'đại_học': 76,
 'biết': 75,
 'làm': 75,
 'điểm': 72,
 'ngày': 71,
 'nhiều': 71,
 'ra': 71,
 'anh': 69,
 'như': 67,
 'thầy': 66,
 'chưa': 65,
 'nên': 64,
 'xin': 64,
 'lớp': 63,
 'muốn': 63,
 'ai': 62,
 '3': 61,
 'cần': 61,
 'đó': 61,
 'chỉ': 59,
 'sẽ': 59,
 'vận_tải': 59,
 'khi': 56,
 'tớ': 55,
 'cả': 54,
 'nhau': 54,
 'rất': 54,
 'sau': 53,
 'cái': 51,
 'hay': 51,
 'mới': 51,
 'thôi': 51,

Xét độ tạp chất của từng từ trong từ điển:

In [75]:
def gini_score(n_class_samples : list[int]) -> float:
    gini : float = 0
    n_samples : int = 0
    for n in n_class_samples:
        n_samples += n
    for n in n_class_samples:
        gini += (n / n_samples) ** 2
    
    return 1 - gini

In [80]:
gini_score_vocabulary : dict[str, float] = {}
for token in vocabulary:
    gini_score_vocabulary[token] = gini_score([
        positive_frequency[token],
        negative_frequency[token],
        neutral_frequency[token]
    ])

gini_score_vocabulary = dict(sorted(gini_score_vocabulary.items(), key=lambda x:x[1]))


In [81]:
gini_score_vocabulary

{'00': 0.0,
 '01': 0.0,
 '024': 0.0,
 '0396': 0.0,
 '06': 0.0,
 '068': 0.0,
 '06đ': 0.0,
 '07h30': 0.0,
 '0918': 0.0,
 '0936': 0.0,
 '0973': 0.0,
 '0979': 0.0,
 '0981': 0.0,
 '0986': 0.0,
 '1000': 0.0,
 '100m': 0.0,
 '100điểm': 0.0,
 '101': 0.0,
 '102a2': 0.0,
 '1045': 0.0,
 '10700': 0.0,
 '108': 0.0,
 '1096': 0.0,
 '10d': 0.0,
 '10ml': 0.0,
 '10tp': 0.0,
 '10tr': 0.0,
 '110': 0.0,
 '1120': 0.0,
 '1134': 0.0,
 '11r': 0.0,
 '120': 0.0,
 '1256': 0.0,
 '127': 0.0,
 '129m2': 0.0,
 '12h': 0.0,
 '12k': 0.0,
 '139': 0.0,
 '13m2': 0.0,
 '13ml': 0.0,
 '14': 0.0,
 '144hz': 0.0,
 '1470': 0.0,
 '14h30': 0.0,
 '15000': 0.0,
 '15h': 0.0,
 '165hz': 0.0,
 '16h': 0.0,
 '17h30': 0.0,
 '17h45': 0.0,
 '17đ': 0.0,
 '18h': 0.0,
 '18h30': 0.0,
 '18h30h': 0.0,
 '18đ': 0.0,
 '190': 0.0,
 '1900': 0.0,
 '1918': 0.0,
 '192': 0.0,
 '1945': 0.0,
 '1946': 0.0,
 '1949': 0.0,
 '1952': 0.0,
 '1956': 0.0,
 '1962': 0.0,
 '1966': 0.0,
 '1968': 0.0,
 '1972': 0.0,
 '1973': 0.0,
 '1977': 0.0,
 '197m2': 0.0,
 '1982': 0.0,
 '1

In [109]:
THRESHOLD = 1
MIN_SAMPLES = 3

features_vocabulary : dict[str, str] = {} 

for key in gini_score_vocabulary:
    total_samples = positive_frequency[key] + negative_frequency[key] + neutral_frequency[key]
    if gini_score_vocabulary.get(key) <= THRESHOLD and total_samples >= MIN_SAMPLES:
        feq_max = max([
            positive_frequency[key],
            negative_frequency[key],
            neutral_frequency[key],
        ])
        if feq_max == positive_frequency[key]:
            features_vocabulary[key] = "Positive"
        elif feq_max == negative_frequency[key]:
            features_vocabulary[key] = "Negative"
        else:
            features_vocabulary[key] = "Neutral"


In [110]:
for key in features_vocabulary:
    if features_vocabulary.get(key) == "Negative":
        print(key)

bcs
bán_nước
bẩn
bốt
bớt
bức_xúc
chiếm
cmn
cư_xử
cất
cắt
cốp
deadline
dong
dí
giang_hồ
giáo_vụ
grab
gà
gậy
hình_như
khâu
lão
lùa
lũ
may_ra
ngoại_hình
ngón
nhan_sắc
nón
nọ
phản_ánh
quát
soát
sập
thiếu_thốn
trườg
tệ
tổn_thương
web
xước
áo_mưa
ó
đơn_phương
đực
ốm
1k
dắt
chửi
chặn
im
kêu
tắc
đếm
2k
già
gọn
kẹt
phi
phụ
yếu
đổ
nhà_xe
100k
60
9tr
chật
chốn
gắt
nhân_phẩm
thừa
vô_duyên
đắt
bà
vụ
20k
40
bách_khoa
chương
chịu_đựng
chở
cấm
dừng
hậu
học_kỳ
hổ
khó_chịu
làm_ăn
phóng
sửa
thi_công
tụi
xe_đạp
giải_quyết
leo
kém
cháu
a7
chú
mệt
xe
54
70
90
a6
bố_mẹ
bộ_phận
canh
cs1
cuốn
cán_sự
cổ
cửa
dương
fes
gom
huỷ
hài_lòng
hành_chính
khổ
kéo
liên_thông
lì_xì
lườm
lịch_sự
lỗi
mơ_ước
mạnh_dạn
nuốt
nếu_như
oi
rbc
rác
rưỡi
suy
sáng_rực
sóng
sư_phạm
tay_lái
thậm_chí
thế_thì
trách
trốn
trộm
tát
tường
tỏ
từ_bỏ
tự_dưng
u
vch
vàng
vắng
vỏ
vỡ
vững
ám_ảnh
điên
đuổi
định
nặng
trả
thái_độ
cay
chúng_nó
cụ
học_hành
khuân
khách
la
nát
nắng
đèn
bác
bãi
tý
ý_thức
giữ
buồn_cười
béo
chiến_lược
giường
gây
hiến
làm_sao
mù

# 2. Xét n-gram = 2:

In [ ]:
positive_frequency : dict[str, int] = {}
for token in vocabulary:
    positive_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Positive"].values:
    for token in sen:
        positive_frequency[token] += 1

negative_frequency : dict[str, int] = {}
for token in vocabulary:
    negative_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Negative"].values:
    for token in sen:
        negative_frequency[token] += 1

neutral_frequency : dict[str, int] = {}
for token in vocabulary:
    neutral_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Neutral"].values:
    for token in sen:
        neutral_frequency[token] += 1
